In [1]:
import numpy as np
import math
from qiskit.circuit import QuantumCircuit,QuantumRegister
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
# from AdaptVQE_.My_adapt import QubitAdapt,MyAdaptVQE
from AdaptVQE_.MyQubit_Adapt_VQE import QubitAdapt
from AdaptVQE_.My_adapt import MyAdaptVQE
# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [2]:
from qiskit.quantum_info import SparsePauliOp
#a = MyAdaptVQE(ES_problem=problem)
b = QubitAdapt(ES_problem=problem)


logger init done!
完整池320,Propotion=0.25,实际池80
开始初始化:挑选第一个算符...
初始化结果:第16项被选定,此项梯度最大,为0.24593734848354704


In [3]:
b.run_slover()

------------第1轮正在进行中--------------
**目前已有[16]**
initial point是:[0.]
第1轮的优化结果是：[0.10284050659552485]
目前这是第1轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.05506444357458454
目前还没有达到收敛！
第1轮中梯度最大项为第15项,已被选入算符池...
------------第2轮正在进行中--------------
**目前已有[16, 15]**
initial point是:[0. 0.]
第2轮的优化结果是：[0.0452377927371601, 0.10160219313092816]
目前这是第2轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.0474583216511651
目前还没有达到收敛！
第2轮中梯度最大项为第11项,已被选入算符池...
------------第3轮正在进行中--------------
**目前已有[16, 15, 11]**
initial point是:[0. 0. 0.]
第3轮的优化结果是：[-0.003684828639179925, 0.04526172716030873, 0.10144955320131994]
目前这是第3轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.04099117424871396
目前还没有达到收敛！
第3轮中梯度最大项为第25项,已被选入算符池...
------------第4轮正在进行中--------------
**目前已有[16, 15, 11, 25]**
initial point是:[0. 0. 0. 0.]
第4轮的优化结果是：[-0.0037016886129634555, 0.04524721930935455, 0.10143857155687572, 0.007315997798357803]
目前这是第4轮,正在准备挑选下一块算符,并验证是否收敛...
没有达到收敛标准,标准为0.001,当前值为0.03682624644096892
目前还没有达到收敛！
第4轮中梯度最大项为第45项

In [4]:
b.adapt_ansatz
b.

[Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_16[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_15[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_11[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_25[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_45[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_60[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_74[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_39[0])]),
 Instruction(name='EvolvedOps', num_qubits=12, num_clbits=0, params=[ParameterVectorElement(term_36[0])]),
 Instruction(name='EvolvedOps', num_q

In [ ]:
from qiskit.quantum_info import Statevector
qc = QuantumCircuit(b.n_qubit)
qc.append(a.excitation_pool_instruction[1],range(b.n_qubit))
Statevector(data=qc.bind_parameters([0.0]))
vqe = VQE(ansatz=qc,estimator=Estimator(),optimizer=SLSQP())
vqe.compute_minimum_eigenvalue(b.hamiltonian)


In [ ]:
from qiskit.primitives.utils import bound_circuit_to_instruction
bound_circuit_to_instruction(circuit=qc)

In [ ]:
b.paulistring_withoutz_instruction[2]

In [ ]:
b.run()

In [ ]:
qc = QuantumCircuit(b.n_qubit)
qc.append(b.init_state_hf,range(b.n_qubit))
# qc.append(b.paulistring_withoutz_instruction[1],range(b.n_qubit))
qc.append(a.excitation_pool_instruction[2],range(b.n_qubit))
vqe = VQE(estimator=Estimator(),ansatz=qc,optimizer=SLSQP(),initial_point=np.zeros(qc.num_parameters))



In [ ]:
vqe.compute_minimum_eigenvalue(b.hamiltonian)

In [ ]:
from AdaptVQE_.My_adapt import MyAdaptVQE
a = MyAdaptVQE(ES_problem=problem)
a.run_tmp()

In [ ]:
#a.slover.ansatz.append(b.paulistring_withoutz_instruction[3],range(4))
a.slover.initial_point=np.zeros(2)
a.slover.compute_minimum_eigenvalue(a.hamiltonian)

In [ ]:
 def evaluate_energy(parameters: np.ndarray) -> np.ndarray | float:
            nonlocal eval_count

            # handle broadcasting: ensure parameters is of shape [array, array, ...]
            parameters = np.reshape(parameters, (-1, num_parameters)).tolist()
            batch_size = len(parameters)

            try:
                job = self.estimator.run(batch_size * [ansatz], batch_size * [operator], parameters)
                estimator_result = job.result()
            except Exception as exc:
                raise AlgorithmError("The primitive job to evaluate the energy failed!") from exc

            values = estimator_result.values

            if self.callback is not None:
                metadata = estimator_result.metadata
                for params, value, meta in zip(parameters, values, metadata):
                    eval_count += 1
                    self.callback(eval_count, params, value, meta)

            energy = values[0] if len(values) == 1 else values

            return energy

        return evaluate_energy

In [ ]:
a = None
a == None

In [ ]:
from qiskit.quantum_info import Statevector
Statevector(bound_circuit_to_instruction(circ))